First: Change Runtime to GPU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/IPE_detection

In [ ]:
pwd


**Step 01: Installing the Packages**

In [ ]:
%%capture
!pip install super-gradients==3.1.0
!pip install imutils
!pip install roboflow
!pip install pytube --upgrade
!pip install super_gradients

**Step 02: Importing all the required libraries**

In [ ]:
from super_gradients.training import Trainer
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val


In [ ]:
from IPython.display import clear_output

In [ ]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

In [ ]:
from super_gradients.training import models

**Step 03: Setting the checkpoint directory and experiment name | Instantiated the trainer**

In [ ]:
CHECKPOINT_DIR = 'checkpoints'
trainer = Trainer(experiment_name='ppe_yolonas_run', ckpt_root_dir=CHECKPOINT_DIR)

**Step 04: Exporting the Dataset from Roboflow into the Google Colab Notebook**

In [ ]:
pwd

**Step 05: Load the dataset parameters into a dictionary**

In [ ]:

dataset_params = {
    'data_dir':'/content/drive/MyDrive/IPE_detection/PPE-Detection-2',
    'train_images_dir':'/content/drive/MyDrive/IPE_detection/PPE-Detection-2/train/images',
    'train_labels_dir':'/content/drive/MyDrive/IPE_detection/PPE-Detection-2/train/labels',
    'val_images_dir':'/content/drive/MyDrive/IPE_detection/PPE-Detection-2/valid/images',
    'val_labels_dir':'/content/drive/MyDrive/IPE_detection/PPE-Detection-2/valid/labels',
    'test_images_dir':'/content/drive/MyDrive/IPE_detection/PPE-Detection-2/test/images',
    'test_labels_dir':'/content/drive/MyDrive/IPE_detection/PPE-Detection-2/test/labels',
    'classes': ['Drill Area','Electrical Panel','Gloves','Harness','Improper Safety Glasses','Mask','No Gloves','No Safety Glasses','No Safety Helmet','No Safety Jacket','No Safety Shoes','Person','Safety Glasses','Safety Helmet','Safety Shoes','Safety Vest']
}

**Step 06: Pass the values for dataset_params into the dataset_params argument as shown below.**

In [ ]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)
val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

clear_output()

**Step 07: Inspect the Dataset Defined Earlier**

In [ ]:
train_data.dataset.transforms

In [ ]:
train_data.dataset.dataset_params['transforms'][1]

In [ ]:
train_data.dataset.dataset_params['transforms'][1]['DetectionRandomAffine']['degrees'] = 10.42

**Step 08: Plot a batch of training data with their augmentations applied to see what they look like**

In [ ]:
train_data.dataset.plot()

**Step 09: Instantiating the model**

In [ ]:
model = models.get('yolo_nas_m',
                   num_classes=len(dataset_params['classes']),
                   pretrained_weights="coco"
                   )

**Step 10: Define metrics and training parameters**

In [ ]:
train_params = {
    # ENABLING SILENT MODE
    'silent_mode': True,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    # TRAINING FOR 20 EPOCHS
    "max_epochs": 20,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        # NOTE: num_classes needs to be defined here
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            # NOTE: num_classes needs to be defined here
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    # mean average precision
    "metric_to_watch": 'mAP@0.50'
}


**Download the Demo Videos**

In [ ]:
!gdown "https://drive.google.com/uc?id=1crFwrpMF1OlaJ0ZCZjBNRo9llLEVR8VQ&confirm=t"

In [ ]:
!gdown "https://drive.google.com/uc?id=1cTIBNQ1R_7JAOURVv9cJ6P935ym_IkZ0&confirm=t"

**Step 11: Training the model**

In [ ]:
trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)

**Step 12: Get the best trained model**

In [ ]:
best_model = models.get('yolo_nas_m',
                        num_classes=len(dataset_params['classes']),
                        checkpoint_path="/content/drive/MyDrive/IPE_detection/checkpoints/ppe_yolonas_run/ckpt_best.pth")

**Step 13:  Evaluating the best trained model on the test set**

In [ ]:

trainer.test(model=best_model,
            test_loader=test_data,
            test_metrics_list=DetectionMetrics_050(score_thres=0.1,
                                                   top_k_predictions=300,
                                                   num_cls=len(dataset_params['classes']),
                                                   normalize_targets=True,
                                                   post_prediction_callback=PPYoloEPostPredictionCallback(score_threshold=0.01,
                                                                                                          nms_top_k=1000,
                                                                                                          max_predictions=300,
                                                                                                          nms_threshold=0.7)
                                                  ))

**Step 14: Predicting with the best model**

In [ ]:
img_url = '/content/drive/MyDrive/IPE_detection/test-img.jpg'
best_model.predict(img_url).show()

Prediction on GLASSES

**Saving the best model**

In [ ]:
# Save the best model
torch.save(best_model, "/content/drive/MyDrive/IPE_detection/best_model.pth")

**Step 15: Testing on a video**

In [ ]:
input_video_path = f"/content/drive/MyDrive/IPE_detection/demo.mp4"
output_video_path = "/content/drive/MyDrive/IPE_detection/detections1.mp4"

In [ ]:

device = 'cuda' if torch.cuda.is_available() else "cpu"

In [ ]:
best_model.to(device).predict(input_video_path).save(output_video_path)

testing in real time

In [ ]:
!pip install torch

In [ ]:
import numpy as np
import cv2
import torch
cap = cv2.VideoCapture(0)
# model = torch.hub.load('best_model', 'custom', path='./best_model.pth', force_reload=True)
best_model.load_state_dict(torch.load('best_model.pth'))
best_model.eval()
while cap.isOpened():
    ret, frame = cap.read()

    # Make detections
    results = best_model(frame)

    cv2.imshow('best_model', np.squeeze(results.render()))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
# cv2.destroyAllWindows()